# チュートリアル

## 組合せ最適化とは

組合せ最適化とは

1. **多数の組合せの選択肢**の中から
2. **制約条件**を満たしつつ、
3. **目的関数**の一番良い組合せを

選び出す問題（組合せ最適化問題）を解く技術です。  
製造・物流スケジューリング等に幅広く利用されています。

以下に身近な組合せ最適化問題の例を記載します。  
例1）遠足のお菓子選択

1. **たくさんのお菓子**の中から
2. **合計300円以内**で、
3. **自分が最も満足**のいく

お菓子の組合せを選ぶ。  
例2）出張の移動経路選択

1. **複数の移動経路**の中から
2. **移動時間4時間以内**で、
3. **自分が最も満足**のいく

経路の組合せを選ぶ。

本テーマでは、演習問題を通じて組合せ最適化技術を体験して頂きます。

## 演習問題

会社$100$社を営業$3$人が訪問し、プレゼンを実施します。各営業のカバンの容量は$700$です。  
$i$番目の会社の番号$CID_i$、位置$\left(X_i, Y_i\right)$、プレゼン希望時間帯$\left[EPT_i, LPT_i\right]$、プレゼン時間$PT_i$、プレゼン資料重量$PW_i$が与えられたとき、以下の評価関数が最小となるような各営業の会社訪問順序を求めてください。  
ただし、各会社には営業1人が必ず1回のみ訪問します。

$$
    \text{評価関数} = \text{総業務時間} + \text{ペナルティ}
$$

各営業のスタート・ゴール地点を$(0, 0)$、移動距離をユークリッド距離（=業務時間）とします。  
会社$i$のプレゼン開始時間を$t_i$、営業$k$が訪問する会社のプレゼン資料の総重量を$spw_k$としたとき、

- プレゼン希望時間帯よりも早くプレゼンを開始した場合、その時間をペナルティとして追加
    $$
        \text{ペナルティ} = \max{\left\{EPT_i - t_i, 0\right\}}
    $$
- プレゼン希望時間帯よりも遅くプレゼンが終了した場合、その時間の2倍をペナルティとして追加
    $$
        \text{ペナルティ} = 2\times\max{\left\{\left(t_i+PT_i\right)-FPT_i, 0\right\}}
    $$
- 訪問する会社のプレゼン資料の総重量よりカバン容量が小さかった場合、不足分の10倍をペナルティとして追加
    $$
        \text{ペナルティ} = 10\times\max{\left\{spw_k - 700, 0\right\}}
    $$

します。

上記は、

1. **各営業がすべての会社をちょうど1回ずつ訪問するような経路**の中から
2. **制約条件**を満たしつつ、
3. **評価関数**が最小となる

各営業の経路を求める問題と言い換えることができます。  
この問題は代表的な組合せ最適化問題の一つで、「配送計画問題」と呼ばれています。  
ただし、本演習問題では一般的な配送計画問題とは異なる設定にしていることに注意してください。

## サンプルコード

ここでは、局所探索法を用いた求解を紹介します。

局所探索法とは、与えられた初期解から始めて、現在解$\boldsymbol{\sigma}$の近傍内に$\boldsymbol{\sigma}$より良い解があればそれに置き換える、という操作を繰り返し実施します。  
現在解から近傍内の解を生成する操作は「近傍操作」と呼ばれます。また、最終的な解は近傍内に自分より良い解が存在しない状態となり、この解は「局所最適解」と呼ばれます。

以下に、局所探索法のアルゴリズムを示します。

1. 初期解$\boldsymbol{\sigma}$を生成する。
2. 改善がなくなるまで以下を実施する。
   1. 近傍解$\boldsymbol{\sigma}^{\prime}$を生成する。
   2. 現在解$\boldsymbol{\sigma}$より近傍解$\boldsymbol{\sigma}^{\prime}$の方が良い場合、現在解を$\boldsymbol{\sigma}^{\prime}$に置き換える。
3. 現在解$\boldsymbol{\sigma}$を出力する。

### パッケージ読み込み

In [1]:
# パッケージ読み込み
from __future__ import annotations
import pandas as pd
import math
import random
import copy

# 乱数シードを固定
random.seed(20220428)

ModuleNotFoundError: No module named 'pandas'

### 入力データ読み込み

In [2]:
# 入力データ読み込み
df = pd.read_csv("../data/in/companies.csv", index_col=0)
df.head()

,X,Y,EPT,FPT,PT,PW
CID,,,,,,
1,5,15,118,278,90,20
2,-7,-15,2573,2733,90,10
3,-15,35,2119,2279,90,20
4,-17,2,209,369,90,10
5,25,10,1546,1706,90,30


### 定数の定義

In [3]:
# 定数
# 会社リスト
COMPANIES = df.index.tolist()
# 会社数
COMPANY_CNT = len(COMPANIES)
# 営業リスト
SALES = [1, 2, 3]
# 営業の人数
SALES_CNT = len(SALES)
# 営業のカバン容量
SALES_CAPACITY = 700

### 評価関数値を取得する関数を定義

In [4]:
def get_obj(res: dict[int, list[int]]) -> float:
    """
    解の評価関数値を取得する.

    Args:
        res (dict[int, list[int]]): 解

    Returns:
        float: 評価関数値
    """
    obj = 0
    for k in SALES:
        t = 0
        penalty = 0
        spw = 0
        # スタート地点（原点）
        prev_x, prev_y = 0, 0
        for i in res[k]:
            # 業務時間
            x, y = df.at[i, "X"], df.at[i, "Y"]
            diff_x, diff_y = abs(x - prev_x), abs(y - prev_y)
            t += math.sqrt(diff_x*diff_x + diff_y*diff_y)

            # プレゼン時間に関するペナルティ
            ept, fpt, pt = df.at[i, "EPT"], df.at[i, "FPT"], df.at[i, "PT"]
            penalty += max(ept-t, 0)
            penalty += 2*max(0, (t+pt)-fpt)

            # プレゼン時間を追加
            t += pt

            # 訪問する会社のプレゼン資料の総重量
            pw = df.at[i, "PW"]
            spw += pw

            prev_x, prev_y = x, y
        # 最後に訪問する会社からゴール地点（原点）への業務時間
        x, y = 0, 0
        diff_x, diff_y = abs(x - prev_x), abs(y - prev_y)
        t += math.sqrt(diff_x*diff_x + diff_y*diff_y)

        # カバン容量に関するペナルティ
        penalty += 10*max(0, spw-SALES_CAPACITY)

        obj += t + penalty
    return obj

### 初期解の作成

初期解として、各営業に会社を均等に分けます。

In [5]:
# 初期解作成
# res[k]：営業kの会社訪問順リスト
res = {k: [] for k in SALES}
# 会社を営業人数分均等に分ける
k = 0
for i in COMPANIES:
    res[k+1].append(i)
    k = (k+1) % SALES_CNT
obj = get_obj(res)

# 結果出力
for k in SALES:
    print(f"{k}: {res[k]}")
print(obj)

1: [1, 4, 7, 10, 13, 16, 19, 22, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55, 58, 61, 64, 67, 70, 73, 76, 79, 82, 85, 88, 91, 94, 97, 100]
2: [2, 5, 8, 11, 14, 17, 20, 23, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56, 59, 62, 65, 68, 71, 74, 77, 80, 83, 86, 89, 92, 95, 98]
3: [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33, 36, 39, 42, 45, 48, 51, 54, 57, 60, 63, 66, 69, 72, 75, 78, 81, 84, 87, 90, 93, 96, 99]
259592.1405093539


### 近傍操作の作成

近傍操作として、以下を作成しておきます。

- swap: 営業$k_1$の$i_1$番目に訪問する会社と営業$k_2$の$i_2$番目に訪問する会社を入替えます。

In [6]:
def swap(res: dict[int, list[int]]):
    """
    swap操作を実施する.

    Args:
        res (dict[int, list[int]]): 解
    """
    # 営業k1のi1番目に訪問する会社と営業k2のi2番目に訪問する会社を入替える
    k1, k2 = random.choices(SALES, k=2)
    i1 = random.randrange(len(res[k1]))
    i2 = random.randrange(len(res[k2]))
    # 同じ営業の同じ会社を入替える場合、何もしない
    if k1 == k2 and i1 == i2:
        return
    res[k1][i1], res[k2][i2] = res[k2][i2], res[k1][i1]

### 局所探索法の実施

局所探索法により解の改善を図ります。

In [7]:
SEARCH_CNT = int(1e3)
for _ in range(SEARCH_CNT):
    nbhd_res = copy.deepcopy(res)
    # 近傍操作
    swap(nbhd_res)
    nbhd_obj = get_obj(nbhd_res)
    # 改善した場合、解を更新
    if nbhd_obj < obj:
        res = nbhd_res
        obj = nbhd_obj

# 結果出力
for k in SALES:
    print(f"{k}: {res[k]}")
print(obj)

1: [80, 4, 92, 45, 17, 13, 53, 43, 100, 76, 55, 62, 81, 58, 98, 84, 99, 28, 90, 20, 72, 77, 66, 48, 32, 87, 71, 36, 96, 41, 22, 39, 10, 89]
2: [1, 63, 69, 11, 74, 86, 16, 68, 82, 94, 93, 35, 12, 61, 75, 65, 73, 8, 83, 88, 56, 50, 64, 40, 23, 91, 2, 21, 26, 44, 34, 78, 37]
3: [59, 46, 57, 38, 7, 30, 9, 31, 27, 5, 95, 49, 25, 42, 24, 19, 47, 54, 85, 52, 70, 97, 67, 51, 18, 29, 3, 6, 15, 33, 14, 79, 60]
97947.1263979882


### 解の確認

In [8]:
# 結果チェック
# 営業の存在確認
for k in res.keys():
    if not k in SALES:
        print(f"営業{k}は存在しません。")
# 会社の存在確認
for v in res.values():
    for i in v:
        if not i in COMPANIES:
            print(f"会社{i}は存在しません。")
# 会社の訪問数確認
company_cnt = sum(len(v) for v in res.values())
if company_cnt != COMPANY_CNT:
    print("会社の訪問数が会社数と一致しません。")
# 会社の訪問過不足確認
for i in COMPANIES:
    has_company = False
    for v in res.values():
        if i in v:
            has_company = True
    if not has_company:
        print(f"会社{i}を訪問していません。")

### 結果ファイルの作成

In [9]:
# 結果ファイル作成
l = []
for k in SALES:
    for i in res[k]:
        l.append([k, i])
sub = pd.DataFrame(l, columns=("SID", "CID"))
sub.to_csv("../data/out/sample_res.csv", index=None)